# TP 1 - NLP
| Nome | Matrícula |
|------|-----------|
| João Vítor Santana Depollo | 2021039751 |

### Setup dependencies & Configuration

In [ ]:
%pip install gensim
%pip install numpy


Defaulting to user installation because normal site-packages is not writeable
DEPRECATION: distro-info 1.1build1 has a non-standard version number. pip 24.1 will enforce this behaviour change. A possible replacement is to upgrade to a newer version of distro-info or contact the author to suggest that they release a version with a conforming version number. Discussion can be found at https://github.com/pypa/pip/issues/12063

[notice] A new release of pip is available: 24.0 -> 24.3.1
[notice] To update, run: pip install --upgrade pip
Defaulting to user installation because normal site-packages is not writeable
DEPRECATION: distro-info 1.1build1 has a non-standard version number. pip 24.1 will enforce this behaviour change. A possible replacement is to upgrade to a newer version of distro-info or contact the author to suggest that they release a version with a conforming version number. Discussion can be found at https://github.com/pypa/pip/issues/12063

[notice] A new release of pip is a

In [26]:
from gensim.models import Word2Vec
import numpy as np
from gensim.models.word2vec import Text8Corpus



In [27]:
# To chose the model, you can pass "CBOW" or "SKIPGRAM" to the `model_type` variable.

dataset_path = "questions-words.txt"
model = "SKIPGRAM"
text8_path="text8"

def generate_all_parameters(model_type):
    vectors_sizes = [10, 20,30]
    windows = [3, 5, 10]
    ALPHA = 0.001
    negative = [2, 5, 10]
    sg = 0 if model_type == "CBOW" else 1
    
    parameters = []
    for vector_size in vectors_sizes:
        for window in windows:
            for neg in negative:
                parameters.append({
                    'vector_size': vector_size,
                    'window': window,
                    'sg': sg,
                    'alpha': ALPHA,
                    'negative': neg
                })
    return parameters

parameters = generate_all_parameters(model)


In [ ]:
           
def cosine_distance(v1, v2):
    return (1 - np.dot(v1, v2) / (np.linalg.norm(v1) * np.linalg.norm(v2)))


def evaluate_model(model, dataset_path):
    total_distance = 0
    count = 0
    
    with open(dataset_path, 'r') as f:
        for line in f:
            if line_is_invalid(line):
                continue
            words = line.strip().split()
            if all(word in model.wv for word in words):
                # Operação de vetores
                result_vector = model.wv[words[1]] - model.wv[words[0]] + model.wv[words[2]]
                expected_vector = model.wv[words[3]]
                
                # Distância do cosseno
                distance = cosine_distance(expected_vector, result_vector)
                total_distance += distance
                count += 1
    return total_distance / count if count > 0 else float('inf')

def line_is_invalid(line):
    # Não considerar os grupos de analogias
    return line.startswith(':') or line == '\n'
def train_model(model_params: dict, corpus: list) -> Word2Vec.__class__:
    model = Word2Vec(
        sentences=corpus,
        vector_size=model_params['vector_size'],
        window=model_params['window'],
        sg=model_params['sg'],
        alpha=model_params['alpha'], 
        negative=model_params['negative'],
        min_count=5,
        sample=1e-3,
        workers=8,
        epochs=5
    )
    
    return model

# Função principal
def train_and_evaluate(dataset_path, parameters):
    corpus = Text8Corpus("text8")
    results = []
    min_distance = float('inf')
    best_model = Word2Vec()
    best_params = {}

    for param in parameters:
        print(f"Treinando modelo com parâmetros: {param}")
        model = train_model(param, corpus)
        
        # Avaliação do modelo
        distance = evaluate_model(model, dataset_path)
        results.append((param, distance))

        if ( distance < min_distance ):
            min_distance = distance
            best_model = model
            best_params = param

        print(f"Distância média: {distance}")

    
    print("\nMelhor modelo:", best_params)
    print("Menor distância média:", min_distance)

    return best_model, best_params


best_model, best_params = train_and_evaluate(dataset_path, parameters)

def test_analogies(model, test_cases):
    print("\nTestando analogias experimentais:")
    for analogy in test_cases:
        word_a, word_b, word_c = analogy[:3]
        if all(word in model.wv for word in [word_a, word_b, word_c]):
            result_vector = model.wv[word_b] - model.wv[word_a] + model.wv[word_c]
            closest_word = model.wv.most_similar([result_vector], topn=1)[0][0]
            print(f"{word_a} -> {word_b}, {word_c} -> {closest_word}")
            print( model.wv.most_similar([result_vector], topn=5))
            
            
test_cases = [
    ['boy', 'brother', 'girl'],
    ["man", "king", "woman"],  
    ["paris", "france", "berlin"],     
    ["man", "actor", "woman"],
    ["man", "husband", "woman"],
    ["day", "sun", "night"]
]
test_analogies(best_model, test_cases)

print(cosine_distance(best_model.wv['man'], best_model.wv['woman'])) 

Treinando modelo com parâmetros: {'vector_size': 10, 'window': 3, 'sg': 1, 'alpha': 0.001, 'negative': 2}
Distância média: 0.5500563087886103
Treinando modelo com parâmetros: {'vector_size': 10, 'window': 3, 'sg': 1, 'alpha': 0.001, 'negative': 5}
Distância média: 0.2599527496569966
Treinando modelo com parâmetros: {'vector_size': 10, 'window': 3, 'sg': 1, 'alpha': 0.001, 'negative': 10}
Distância média: 0.16027841196572018
Treinando modelo com parâmetros: {'vector_size': 10, 'window': 5, 'sg': 1, 'alpha': 0.001, 'negative': 2}
Distância média: 0.4776142473064163
Treinando modelo com parâmetros: {'vector_size': 10, 'window': 5, 'sg': 1, 'alpha': 0.001, 'negative': 5}
Distância média: 0.2123747300222455
Treinando modelo com parâmetros: {'vector_size': 10, 'window': 5, 'sg': 1, 'alpha': 0.001, 'negative': 10}
Distância média: 0.13986569678537933
Treinando modelo com parâmetros: {'vector_size': 10, 'window': 10, 'sg': 1, 'alpha': 0.001, 'negative': 2}
Distância média: 0.3602312934069362
T